In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU, RNN

Using TensorFlow backend.


In [3]:
data = pd.read_csv('../input_data/GoogleStocks.csv')
data = data.drop('date',1)

In [4]:
data = data.iloc[1:]
data['avg'] = data[['low', 'high']].mean(axis=1)

In [5]:
data.head()

,close,volume,open,high,low,avg
1,1184.62,1894639,1198.53,1202.8300,1176.720,1189.77500
2,1193.00,1493841,1196.93,1206.3975,1187.040,1196.71875
3,1205.50,1668910,1226.32,1230.0000,1202.825,1216.41250
4,1231.54,1195899,1216.00,1231.7900,1213.150,1222.47000
5,1223.97,2089367,1197.35,1227.1400,1196.170,1211.65500


In [6]:
sc = MinMaxScaler(feature_range = (0, 1))
scaled_data = sc.fit_transform(data)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [7]:
scaled_data[0]

array([0.86049961, 0.25082533, 0.87921667, 0.88187969, 0.87656555,
       0.88231921])

In [8]:
def RNN2(cells,timesteps):
    X_train = []
    y_train = []
    print('RNN cells:',cells,' Time_Steps:',timesteps, 'layers:2')
    for i in range(timesteps, 755):
        X_train.append([scaled_data[i-timesteps:i, 5],scaled_data[i-timesteps:i, 1]])
        y_train.append(scaled_data[i, 0])
    X_train, y_train = np.asarray(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[2], 2))
    model = Sequential()
    model.add(GRU(units = cells, return_sequences = True, input_shape = (X_train.shape[1], 2)))
    model.add(GRU(units = cells))
    model.add(Dense(units = 1))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs = 10, batch_size = 32, verbose=0)
    score = model.evaluate(X_train, y_train, batch_size=32, verbose=0)
    return score

In [9]:
def RNN3(cells,timesteps):
    X_train = []
    y_train = []
    print('RNN cells:',cells,' Time_Steps:',timesteps, 'layers:3')
    for i in range(timesteps, 755):
        X_train.append([scaled_data[i-timesteps:i, 5],scaled_data[i-timesteps:i, 1]])
        y_train.append(scaled_data[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[2], 2))
    model = Sequential()
    model.add(GRU(units = cells, return_sequences = True, input_shape = (X_train.shape[1], 2)))
    model.add(GRU(units = cells, return_sequences = True))
    model.add(GRU(units = cells))
    model.add(Dense(units = 1))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs = 10, batch_size = 32, verbose=0)
    score = model.evaluate(X_train, y_train, batch_size=32, verbose=0)
    return score

In [10]:
cells2 = [30,50,80]
timesteps2 = [20,50,75]
cells3 = [30,50,80]
timesteps3 = [20,50,75]

In [11]:
loss = []
accuracy = []
combination = 1
for cell in cells2:
    for timestep in timesteps2:
        print('Combination:',combination)
        score = RNN2(cell,timestep)
        print('loss:',score[0],' accuracy:',score[1],'\n')
        loss.append(score[0])
        accuracy.append(score[1])
        combination += 1
for cell in cells3:
    for timestep in timesteps3:
        print('Combination:',combination)
        score = RNN2(cell,timestep)
        print('loss:',score[0],' accuracy:',score[1],'\n')
        loss.append(score[0])
        accuracy.append(score[1])
        combination += 1

Combination: 1
RNN cells: 30  Time_Steps: 20 layers:2
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
loss: 0.0030855734889604605  accuracy: 0.0027210884353741495 

Combination: 2
RNN cells: 30  Time_Steps: 50 layers:2
loss: 0.07038790613201493  accuracy: 0.0014184397163120568 

Combination: 3
RNN cells: 30  Time_Steps: 75 layers:2
loss: 0.0689479874967433  accuracy: 0.0014705882352941176 

Combination: 4
RNN cells: 50  Time_Steps: 20 layers:2
loss: 0.008134390057070928  accuracy: 0.0027210884353741495 

Combination: 5
RNN cells: 50  Time_Steps: 50 layers:2
loss: 0.010727757421744895  accuracy: 0.0028368794326241137 

Combination: 6
RNN cells: 50  Time_Steps: 75 layers:2
loss: 0.06921016475569237  accuracy: 0.0014705882352941176 

Combination: 7
RNN cells: 80  Time_Steps: 20 layers:2
loss: 0.0028005417026135894  accuracy: 0.0027210884353741495 

Combination: 8
RNN cells: 80  Time_Steps: 50 layers:2
loss: 0.04215760

In [12]:
loss

[0.0030855734889604605,
 0.07038790613201493,
 0.0689479874967433,
 0.008134390057070928,
 0.010727757421744895,
 0.06921016475569237,
 0.0028005417026135894,
 0.042157601543986206,
 0.06851408494745984,
 0.0033284913064265736,
 0.06772108795847875,
 0.06893967125181327,
 0.0030119206927412628,
 0.06941395941989642,
 0.0603647831334349,
 0.0038760474350835595,
 0.008637544319879079,
 0.06926720660070286]

In [13]:
accuracy

[0.0027210884353741495,
 0.0014184397163120568,
 0.0014705882352941176,
 0.0027210884353741495,
 0.0028368794326241137,
 0.0014705882352941176,
 0.0027210884353741495,
 0.0028368794326241137,
 0.0014705882352941176,
 0.0027210884353741495,
 0.0014184397163120568,
 0.0014705882352941176,
 0.0027210884353741495,
 0.0014184397163120568,
 0.0014705882352941176,
 0.0027210884353741495,
 0.0028368794326241137,
 0.0014705882352941176]